<a href="https://colab.research.google.com/github/mhy1264/taipower/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install autoKeras
!pip install scikit-learn

In [2]:
import autokeras as ak
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import weather
import uva

Using TensorFlow backend


In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
def get_obv_station(gen_station: str) -> str:
    obv_station = pd.read_csv("./data/mini_dist2.csv")
    df = obv_station[obv_station['gen_station'].astype(str).str.contains("{}".format(gen_station))]
    return df['obv_station'].tolist()[0]

In [8]:
def get_uva_station(gen_station: str):
    uva_station = pd.read_csv("./data/mini_dist_adduv.csv")
    uva_station = uva_station[["gen_station", "uv_station" ,"uv_lng" ,"uv_lat"]]
    df = uva_station[uva_station['gen_station'].astype(str).str.contains("{}".format(gen_station))]
    return df['uv_station'].tolist()[0]

In [10]:

solar_daily = pd.read_csv("./data/solar_daily.csv")
station = pd.read_csv("./data/station.csv")


In [11]:

for row in range(1,2):
    df = solar_daily.iloc[:, [row, -1]]
    current_station = df.columns.tolist()[0]
    
    station_weather = weather.get_data(get_obv_station(current_station))
    # station_weather = weather.get_month_data(get_obv_station(current_station, 2017, 1))   
    fileName = current_station.replace("/","\\")

    # 合併發電站度數和天氣觀測站的天氣資訊
    station_weather['date'] = station_weather['date'].astype(str)
    df['date'] = df["date"].astype(str)
    merged_df = pd.merge(station_weather, df, on="date", how="inner")
    merged_df.rename(columns={current_station:"degree"}, inplace=True)

    # 合併 merge_df 和紫外線資料 放到 UVI max 這格內
    uva_station = get_uva_station(current_station)
    uva_data = uva.get_data(uva_station)
    
    merged_df = pd.merge(merged_df, uva_data, on="date", how="inner")

    merged_df["station"] = [current_station]*merged_df.shape[0]
    merged_df.to_csv("temp{}.csv".format(fileName))


/var/folders/v_/kg8g79f94xg0qgb5xbjj03280000gn/T/ipykernel_67031/1307874008.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df["date"].astype(str)


KeyError: 'obv_station'